In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


## Loading and preparing label maps

In [4]:
with open('../../../wsod/metadata/ont_m18/class_names_all.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [5]:
with open('../../results/det_results_concat_30.pkl', 'rb') as fin:
    det_results_concat = pickle.load(fin)

In [6]:
with open('../../results/det_results_merged_30.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

In [7]:
label_to_img = {key: [] for key in mid2name_all}
for key, val in det_results_merged.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [8]:
all_labels = list(label_to_img.keys())

## Visualizing Results

In [9]:
mid2name_extended = {}
extended_classes = set()
for key in all_labels:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('voc','PA'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J'), ('voc/J','PA/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_all[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [10]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [11]:
result_dict = {
    'Concatenated': det_results_concat,
    'Merged': det_results_merged,
}

In [12]:
path_prefix = '../../../../data/dryrun_m18/jpg/' 

In [13]:
model_abv_dict = {
    'coco': 'CO',
    'voc': 'PA',
    'oi': 'OI',
    'ws': 'WS',
    'coco/J': 'CO/J',
    'voc/J': 'PA/J',
    'oi/J': 'OI/J',
    'ws/J': 'WS/J',
}

In [14]:
def show(imgid, source, thresh):
    filename = path_prefix + imgid + '.jpg.ldcc'
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx_extended[det['label'] + '/' + det['model']] for det in detections]
    
    # print([(mid2name_extended[extended_classes[item]], boxes[i]) for i, item in enumerate(label_idx) if scores[i] > thresh])
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index_extended,
      use_normalized_coordinates=True,
      min_score_thresh=thresh-0.001,
      max_boxes_to_draw=1000,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [15]:
root_to_leaf = {}
flag = False
with open('../../../../data/dryrun_m18/parent_children.tab', 'r') as fin:
    for line in fin:
        if not flag:
            flag = True
            continue
        row = line.split()
        if root_to_leaf.get(row[7]) == None:
            root_to_leaf[row[7]] = []
        root_to_leaf[row[7]].append(row[2])


In [17]:
for root_id in ['IC0011WZ7', 'IC0011WS9', 'IC0011WS1', 'IC0011WS1']:
    print(f'**************************** {root_id} ****************************')
    print(root_to_leaf[root_id])
    for leaf_id in root_to_leaf[root_id]:
        if leaf_id in det_results_merged:
            print(leaf_id)
            w = interactive(show, 
                imgid=fixed(leaf_id),
                source=widgets.RadioButtons(options=['Concatenated', 'Merged'], value='Merged'), 
                thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
            )
            w.children[-1].layout.height = '600px'
            display(w)


**************************** IC0011WZ7 ****************************
['HC000Q807', 'IC0011WZB', 'IC0011WZC']
IC0011WZB


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

IC0011WZC


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

**************************** IC0011WS9 ****************************
['HC000Q7ZP', 'IC0011WSS', 'IC0011WSR', 'IC0011WSQ', 'IC0011WSP', 'IC0011WSO']
IC0011WSS


...

interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

IC0011WS5


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…

IC0011WS6


interactive(children=(RadioButtons(description='source', index=1, options=('Concatenated', 'Merged'), value='M…